# 🧠 Executive Intelligence Copilot - Enhanced Colab Deployment

**Run with GPU acceleration + persistent storage!**

---

## 🚀 Quick Start

1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Run all cells**: Ctrl+F9 or Runtime → Run all
3. **Allow Drive access**: When prompted
4. **Enter API keys**: When prompted
5. **Access app**: Click the ngrok URL

---

## ⚡ Performance

| Operation | Local CPU | Colab GPU |
|-----------|-----------|-----------|
| Model Load | 8s | 2s |
| 10-page Doc | 25s | 1.5s |
| **Speedup** | 1x | **15-20x** |

## 💾 Persistent Storage

Your data saves to Google Drive and persists across sessions!

---


## Step 1: Check GPU Availability


In [ ]:
import torch

print("🔍 Checking GPU availability...")
print("=" * 60)

if torch.cuda.is_available():
    print("✅ GPU DETECTED!")
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
    print("\n🚀 You'll get 15-20x faster embeddings!")
else:
    print("⚠️ NO GPU DETECTED")
    print("   Go to: Runtime → Change runtime type → Select GPU")
    print("   Note: GPU gives 15-20x faster performance!")

print("=" * 60)


## Step 2: Mount Google Drive for Persistent Storage

**This ensures your data persists across Colab sessions!**


In [ ]:
from google.colab import drive
import os

print("📂 Mounting Google Drive for persistent storage...")
print("=" * 60)

# Mount Drive
drive.mount('/content/drive', force_remount=False)

# Create persistent directory structure
DRIVE_PATH = '/content/drive/MyDrive/intelligence_copilot_data'
os.makedirs(DRIVE_PATH, exist_ok=True)
os.makedirs(f'{DRIVE_PATH}/faiss', exist_ok=True)
os.makedirs(f'{DRIVE_PATH}/raw', exist_ok=True)

# Set environment variables for persistent paths
os.environ['DB_PATH'] = f'{DRIVE_PATH}/briefs.db'
os.environ['FAISS_PATH'] = f'{DRIVE_PATH}/faiss'
os.environ['RAW_PATH'] = f'{DRIVE_PATH}/raw'

print("\n✅ PERSISTENT STORAGE CONFIGURED")
print("=" * 60)
print(f"📁 Storage location: {DRIVE_PATH}")
print(f"💾 Database: {os.environ['DB_PATH']}")
print(f"🔍 FAISS: {os.environ['FAISS_PATH']}")
print("\n📌 Your data will survive Colab restarts!")
print("=" * 60)

# Check if existing data
if os.path.exists(os.environ['DB_PATH']):
    size = os.path.getsize(os.environ['DB_PATH']) / 1024
    print(f"\n✅ Found existing database ({size:.1f} KB)")
    print("   Your previous meetings will be loaded!")
else:
    print("\n📝 First run - database will be created")

print("=" * 60)


## Step 3: Upload Your Project

Use folder icon on left to upload your entire `intelligence_copilot` folder


In [ ]:
import os

PROJECT_PATH = "/content/intelligence_copilot"

if os.path.exists(PROJECT_PATH):
    os.chdir(PROJECT_PATH)
    print(f"✅ Working directory: {os.getcwd()}")
    print(f"📂 Files found: {len(os.listdir('.'))} items")
else:
    print(f"❌ Project not found at {PROJECT_PATH}")
    print("\n📤 Please upload:  Click folder icon → Upload folder → intelligence_copilot")


## Step 4: Install Dependencies (~2 minutes)


In [ ]:
print("📦 Installing dependencies...")
%pip install -q -r requirements.txt
%pip install -q pyngrok
print("✅ Dependencies installed!")


## Step 5: Configure API Keys


In [ ]:
import os
from getpass import getpass

print("🔑 API KEY CONFIGURATION\n")
print("Select LLM provider:")
print("  1. Gemini (Free tier - Recommended)")
print("  2. OpenAI")
print("  3. Anthropic")

choice = input("\nEnter choice (1/2/3): ") or "1"
provider = {"1": "gemini", "2": "openai", "3": "anthropic"}[choice]
os.environ["LLM_PROVIDER"] = provider

if provider == "gemini":
    print("\n📝 Get free key: https://makersuite.google.com/app/apikey")
    os.environ["GEMINI_API_KEY"] = getpass("Enter Gemini API key: ")
elif provider == "openai":
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")
elif provider == "anthropic":
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter Anthropic API key: ")

print("\n📡 Ngrok Setup")
print("Sign up free: https://dashboard.ngrok.com/signup")
os.environ["NGROK_TOKEN"] = getpass("Enter ngrok authtoken: ")

print(f"\n✅ Configured: {provider.upper()} + Ngrok")


## Step 6: Launch App 🚀

**Run this and click the ngrok URL!** Logs appear below.


In [ ]:
from pyngrok import ngrok
import threading
import subprocess
import time

ngrok.set_auth_token(os.environ["NGROK_TOKEN"])

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port=8501", "--server.headless=true"])

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

print("🚀 Starting Streamlit...")
time.sleep(10)

public_url = ngrok.connect(8501, bind_tls=True)

print("\n" + "="*60)
print("✅ APP IS RUNNING!")
print("="*60)
print(f"\n🌐 URL: {public_url}")
print("\n📝 Usage: Click URL → Create meeting → Upload docs → Generate brief")
print("💾 Data saves to Google Drive automatically")
print("\n🛑 To stop: Click stop button above")
print("="*60)
print("\n📋 LOGS (real-time):")
print("-"*60)

try:
    thread.join()
except KeyboardInterrupt:
    print("\n🛑 Shutting down...")
    ngrok.disconnect(public_url)


In [ ]:
# Monitor GPU
!nvidia-smi

# Check database
import sqlite3
db = os.environ.get('DB_PATH')
if os.path.exists(db):
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM meetings")
    print(f"\n📊 Database: {cursor.fetchone()[0]} meetings")
    conn.close()
